### Teknofest Veri Ön İşleme - 3

Her memenin LCC fotoğrafını yeniden `kompozisyon` bilgisine göre sınıflandır.

Yapılacaklar:

**1 -)**

Kompozisyon bilgisine göre klasörle

**2 -)**

Sadece LCC fotoğrafını bırak

**3 -)**

Her LCC fotoğrafını Kompozisyon bilgisini başa alıp numaralandırarak isimlendir

Kompozisyon Fotoğrafları:

**A ->** 0... 135 

**B ->** 0... 205
  .
  .
  .
  
**D ->** 0... 155

şeklinde olsun.

**4 -)**

Tüm veri ön işlemeyi birleştir

**Not:** Tek bir liste içersinde tutmana gerek yok


### To Do:

**1 -)** 

Organize according to composition information with folders.

**2 -)** 

Only leave the LCC photo.

**3 -)** 

Rename each LCC photo by placing the composition information at the beginning and numbering them.

Composition Photos:

**A ->** 0... 135 

**B ->** 0... 205
  .
  .
  .
  
**D ->** 0... 155

in the above format.

---

#### Step 1

In [1]:
import os
import pandas as pd
import numpy as np
import openpyxl
import shutil
from pathlib import Path
import pydicom
import dicom2jpg
from PIL import Image, ImageChops
import cv2
import time

In [2]:
data = pd.read_excel("siralama.xlsx")
print(data["MEME KOMPOZİSYONU"])

0     D
1     C
2     D
3     C
4     D
5     C
6     B
7     B
8     B
9     C
10    B
Name: MEME KOMPOZİSYONU, dtype: object


In [3]:
data["MEME KOMPOZİSYONU"].unique()

array(['D', 'C', 'B'], dtype=object)

In [4]:
data.head(10)

,HASTANO,BIRADS KATEGORİSİ,MEME KOMPOZİSYONU,KADRAN BİLGİSİ (SAĞ),KADRAN BİLGİSİ (SOL),"Birads Skoru (EK BİLGİ OLARAK VERİLMİŞTİR, YARIŞMADA İSTENMEYECEKTİR)"
0,822670188,BI-RADS0,D,NaN,"[""MERKEZ""]",Kategori 0: Ek değerlendirme tetkiki yapılmalıdır
1,822670189,BI-RADS1-2,C,NaN,NaN,Kategori 1: Negatif (bir sorun yok)
2,822670197,BI-RADS0,D,"[""MERKEZ""]","[""MERKEZ""]",Kategori 0: Ek değerlendirme tetkiki yapılmalıdır
3,822670201,BI-RADS1-2,C,"[""ÜST DIŞ""]","[""ÜST DIŞ""]",Kategori 2: İyi huylu bulgular
4,822670202,BI-RADS1-2,D,NaN,NaN,Kategori 2: İyi huylu bulgular
5,822670206,BI-RADS1-2,C,NaN,NaN,Kategori 1: Negatif (bir sorun yok)
6,836180612,BI-RADS0,B,NaN,"[""ÜST DIŞ""]",Kategori 5: Büyük olasılıkla kanser olabilir -...
7,836180620,BI-RADS0,B,"[""ÜST DIŞ""]",NaN,Kategori 4: Şüpheli bulgular - Biyopsi yapılma...
8,836180632,BI-RADS0,B,"[""ÜST DIŞ""]",NaN,Kategori 5: Büyük olasılıkla kanser olabilir -...
9,836180633,BI-RADS0,C,"[""ALT İÇ""]",NaN,Kategori 4: Şüpheli bulgular - Biyopsi yapılma...


In [5]:
komp_A = "A"
komp_B = "B"
komp_C = "C"
komp_D = "D"

In [6]:
def categorized_on_dict():
    
    data = pd.read_excel("siralama.xlsx") 
    
    k_A = data[data["MEME KOMPOZİSYONU"] == komp_A]
    k_A_folder = [int(isim) for isim in k_A["HASTANO"]]
    
    k_B = data[data["MEME KOMPOZİSYONU"] == komp_B]
    k_B_folder = [int(isim) for isim in k_B["HASTANO"]]
    
    k_C = data[data["MEME KOMPOZİSYONU"] == komp_C]
    k_C_folder = [int(isim) for isim in k_C["HASTANO"]]
    
    k_D = data[data["MEME KOMPOZİSYONU"] == komp_D]
    k_D_folder = [int(isim) for isim in k_D["HASTANO"]]
        
    k_dict = {komp_A:k_A_folder, komp_B:k_B_folder, komp_C:k_C_folder, komp_D:k_D_folder}
    
    return k_dict

In [7]:
file_path = os.getcwd()
categorized_data = categorized_on_dict()
categorized_data

{'A': [],
 'B': [836180612, 836180620, 836180632, 836194603],
 'C': [822670189, 822670201, 822670206, 836180633],
 'D': [822670188, 822670197, 822670202]}

In [8]:
def new_field_list(source_field):
    """
     To avoid code repetition, this was written as a separate function. 
     Its purpose is to append the names of other fixed files, such as LCC.dcm, to the source field end, and to return this as a part of a list.
    """
    
    files = ["LCC.dcm", "LMLO.dcm", "RCC.dcm", "RMLO.dcm"]
    new_files = ["LCC.png", "LMLO.png", "RCC.png", "RMLO.png"] 
    
    field = ""
    new_field = list()
    new_field_png = list()
    
    for file in files:
        field = source_field + "/" + file
        new_field.append(field)
    
    for n_file in new_files:
        field = source_field + "/" + n_file
        new_field_png.append(field)
    
    return (new_field, new_field_png)
    

In [9]:
def change_and_resize(source_field):
    
    images = new_field_list(source_field)[1]
    
    for image in images:
        try:       
            img = cv2.imread(image)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.resize(gray, (224, 224), interpolation=cv2.INTER_AREA)
            img2 = np.zeros([224, 224, 3], dtype=np.uint8) 
            img2[:, :, 0] = gray
            img2[:, :, 1] = gray
            img2[:, :, 2] = gray
            cv2.imwrite(image, img2)
        except FileNotFoundError as e:
            break
        except Exception as e:
            print(e)
            break


In [10]:
def outlier_detection(source_field):
    """
     This was written here to identify files within the patient number folder that are larger than X MB and smaller than X MB in size.
    """
    
    # What should be done with files that exist in Excel but not in the folder?
    
    size = 0

    for path, dirs, files in os.walk(source_field):
        for f in files:
            try:                
                fp = os.path.join(path, f)
                size += os.path.getsize(fp)
            except OSError:
                return False

    #print("Folder size: " + str(size))    
    if (size < 56623104 and size > 52428800):  
        
        return True
    else:
        
        return False
    

In [11]:
def convert(source_field):
    """
     Convert a file with the .dcm extension to the .jpg format.
    """
    
    list0 = new_field_list(source_field)[0]
    list1 = new_field_list(source_field)[1]
    
    for d, j in zip(list0, list1):
        
        #print(l)

        im = pydicom.dcmread(d)
        im = im.pixel_array.astype(float)

        rescaled_image = (np.maximum(im, 0) / im.max()) * 255.0
        final_image = np.uint8(rescaled_image)

        final_image = Image.fromarray(final_image)
        #final_image.show()

        inv_img = ImageChops.invert(final_image)
        os.remove(d)

        inv_img.save(j)      
  

In [12]:
def crop(source_field):
    """
     For each .png format file you handle, come here and crop it before moving. This way, it will be moved as a cropped version.
    """
    
    #convert(kynk)
    
    my_list = new_field_list(source_field)[1]
    
    for l in my_list:
        
        img = Image.open(l)
        # 2364x2964
       
        if ("LCC" in l or "LMLO" in l):
            
            imgCropped = img.crop(box = (0,0,1500,2964))
            os.remove(l)
            imgCropped.save(l)

        elif("RCC" in l or "RMLO" in l):
            
            imgCropped = img.crop(box = (900,0,2364,2964))
            #os.remove(l)
            imgCropped.save(l)
            
        else:
            pass
     

In [13]:
def move(source, destination):
    
    shutil.move(source ,destination)

In [14]:
def find_odetection_convert_crop_change_and_resize_move(categorized_data):
    """
    In the incoming dictionary structure, we are aware of the files for each category. 
    What we need to do is, while iterating over these folder names, place or move each one to the corresponding category we find.
    """
    
    current_directory = os.getcwd()
    source_directory_path = Path(current_directory + "/test")
    destination_directory_path = Path(current_directory + "/Data")

    target_directory_and_dict_column_list = [komp_A, komp_B, komp_C, komp_D]
    
    for value in target_directory_and_dict_column_list:
        #print(value)
        destination_directory = str(destination_directory_path) + "/" + value
        #print("value ->", destination_directory)
    
        for folder in categorized_data[value]:    
    
            source_path = str(source_directory_path) + "/" + str(folder)
            #print("Source ->", source_path)    
            
            if outlier_detection(source_path):
            
                convert(source_path)  # Call the conversion function to convert to PNG format collectively --> convert() will use source_path
                crop(source_path)     # Crop all of them at once when it comes here --> crop() +
                change_and_resize(source_path) # Resize all of them when it comes here --> resize_images()
                move(source_path, destination_directory) # +
            else:
                
                try:
                    shutil.rmtree(source_path)
                    continue
                except FileNotFoundError:
                    continue

---

#### Step 2

In [15]:
def delete(source_field):
    """
     Delete all images except for LCC.png.
    """
    
    images = new_field_list(source_field)[1]

    for image in images:
        
        if not "LCC" in image:
            try:       
                os.remove(image)
            except FileNotFoundError:
                break
        else:
            
            continue
            

In [16]:
def rename_images(destination_directory, k):
    """
    Rename the photos of each patient file by adding the category number's name as a prefix and numbering them sequentially.
    """
    compositions = ["A", "B", "C", "D"]
    
    new_name = ""
    
    global number 
       
    photos = new_field_list(destination_directory)[1]
    
    if k == compositions[0]:

        for photo in photos:
            new_name = f"A{number}"
            
            try:
                os.rename(src=photo, dst=destination_directory + "/" + new_name + ".png")
            except FileNotFoundError:
                break
                
            number = number + 1
    elif k == compositions[1]:

        for photo in photos:
            new_name = f"B{number}"
            
            try:
                os.rename(src=photo, dst=destination_directory + "/" + new_name + ".png")
            except FileNotFoundError:
                break
                
            number = number + 1
    elif k == compositions[2]:

        for photo in photos:
            new_name = f"C{number}"
            
            try:
                os.rename(src=photo, dst=destination_directory + "/" + new_name + ".png")
            except FileNotFoundError:
                break
                
            number = number + 1 
    elif k == compositions[3]:

        for photo in photos:
            new_name = f"D{number}"
            
            try:
                os.rename(src=photo, dst=destination_directory + "/" + new_name + ".png")
            except FileNotFoundError:
                break
                
            number = number + 1
    else:
        
        pass


In [17]:
def find_rename_and_delete(categorized_data):
    """
     Find and rename the photos of each patient file in the Data folder.
    """
    
    current_directory = os.getcwd()
    directory_path = Path(current_directory + "/Data")

    compositions = ["A", "B", "C", "D"]
    
    global number

    for element in compositions:
        
        number = 0
        
        comp_directory = ""
        
        comp_directory = str(directory_path) + "/" + element
        #print(comp_directory)
        
        for column in categorized_data[element]:
            destination_directory = comp_directory + "/" + str(column)
            #print(destination_directory)
            
            #delete(destination_directory)
            rename_images(destination_directory, element)

---

#### Special

In [18]:
def special_change_and_resize(image):
    
    #print(image)
    try:       
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (224, 224), interpolation=cv2.INTER_AREA)
        img2 = np.zeros([224, 224, 3], dtype=np.uint8) 
        img2[:, :, 0] = gray
        img2[:, :, 1] = gray
        img2[:, :, 2] = gray
        cv2.imwrite(image, img2)
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(e)

In [19]:
def resize_images(image):
 
    image = Image.open(image)
    new_image = image.resize((224, 224))
    new_image.save(image)
    #print(image)

In [20]:
def special_resize(source_field):
    
    os.walk(source_field)
    
    for path, dirs, files in os.walk(source_field):
        
        for f in files:               
            fp = os.path.join(path, f)
            #print(fp)

            #change_and_resize_test(fp)
            #resize_images(fp)


In [ ]:
ka = r"C:\Users\erdem\Desktop\teknofest_veri_on_isleme\Data"

start_time = time.time()
special_resize_test(ka)
print("--- %s seconds ---" % (time.time() - start_time))

---

**Step 1**

In [23]:
start_time = time.time()
find_odetection_convert_crop_change_and_resize_move(categorized_data)
print("--- %s seconds ---" % (time.time() - start_time))

--- 36.800759077072144 seconds ---


**Step 2**

In [24]:
start_time = time.time()
find_rename_and_delete(categorized_data)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.028896093368530273 seconds ---


---

### Test

In [83]:
path = r"C:\Users\erdem\Desktop\teknofest_veri_on_isleme\test\822670197"

print(outlier_detection_test(path))

True


In [53]:
ka = r"C:\Users\erdem\Desktop\teknofest_veri_on_isleme\Data"

start_time = time.time()
special_resize_test(ka)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.7163238525390625 saniye ---


---

#### New İdeas

##### take look lcc.png and rcc.png diff

In [108]:
def diff(source_field):
    
    photos = new_field_list(source_field)[1]
    
    lcc_path = photos[0]  # photos[0] -> LCC 
    rcc_path = photos[3]  # photos[3] -> RCC
    
    lcc_img = Image.open(lcc_path)
    rcc_img = Image.open(rcc_path)
    
    difference = ImageChops.difference(lcc_img, rcc_img)
    
    difference.show()

In [48]:
source = r"C:\Users\erdem\Desktop\teknofest_veri_on_isleme\Data\B\836180620"
diff(source)